In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriatic_Arthritis"
cohort = "GSE142049"

# Input paths
in_trait_dir = "../../input/GEO/Psoriatic_Arthritis"
in_cohort_dir = "../../input/GEO/Psoriatic_Arthritis/GSE142049"

# Output paths
out_data_file = "../../output/preprocess/Psoriatic_Arthritis/GSE142049.csv"
out_gene_data_file = "../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE142049.csv"
out_clinical_data_file = "../../output/preprocess/Psoriatic_Arthritis/clinical_data/GSE142049.csv"
json_path = "../../output/preprocess/Psoriatic_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Analyze the dataset to determine availability of gene expression and clinical data

# 1. Gene Expression Data
# Based on the background information, this dataset contains transcriptional data (RNA)
# from CD19+ B cells, which indicates it contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify rows in the sample characteristics dictionary for trait, age, and gender

# For the trait (Psoriatic Arthritis), we can find it in the working_diagnosis field (key 6)
trait_row = 6

# Age information is available in key 2
age_row = 2

# Gender information is available in key 1
gender_row = 1

# 2.2 Create conversion functions for each variable

def convert_trait(value):
    """
    Convert the working_diagnosis value to a binary indicator for Psoriatic Arthritis.
    1 if the patient has Psoriatic Arthritis, 0 otherwise.
    """
    if isinstance(value, str) and ":" in value:
        diagnosis = value.split(":", 1)[1].strip()
        if diagnosis == "Psoriatic Arthritis":
            return 1
        else:
            return 0
    return None

def convert_age(value):
    """
    Convert age string to a numerical value.
    """
    if isinstance(value, str) and ":" in value:
        age_str = value.split(":", 1)[1].strip()
        try:
            return int(age_str)
        except ValueError:
            return None
    return None

def convert_gender(value):
    """
    Convert gender string to binary: 0 for female, 1 for male.
    """
    if isinstance(value, str) and ":" in value:
        gender = value.split(":", 1)[1].strip()
        if gender.upper() == "F":
            return 0
        elif gender.upper() == "M":
            return 1
    return None

# 3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the library function
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the clinical features to a CSV file
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers, I can see they are ILMN_ prefixed identifiers.
# These are Illumina BeadArray probe IDs, not standard human gene symbols.
# Illumina IDs like "ILMN_1343291" need to be mapped to human gene symbols.
# These IDs are specific to Illumina microarray platforms and require mapping.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the relevant columns from the gene annotation dataframe
# The 'ID' column contains probe IDs (e.g., ILMN_...) which match the gene_data index
# The 'Symbol' column contains the human gene symbols we need to map to

# 2. Get gene mapping dataframe by extracting the needed columns
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print("Gene mapping preview:")
print(preview_df(mapping_data))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)
print("After mapping to gene symbols, gene data shape:", gene_data.shape)
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Store the gene data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Re-extract clinical features since we need it for linking
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from endothelial cells derived from circulating progenitors of RA patients"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    print(f"Data is usable. Saving to {out_data_file}")
    unbiased_linked_data.to_csv(out_data_file)
else:
    print("Data is not usable. Not saving linked data file.")